In [ ]:
#import cupy as cp
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
from tqdm import tqdm
from functions import calibration as cal
import pickle
from tqdm import tqdm

# Based on Jäckel (2015)
from py_vollib_vectorized import vectorized_implied_volatility

### Loading Original Dataset

In [ ]:
# Cleaned test dataframes
with open("data generation/test pickles/cleaned_test_chunks.pkl", "rb") as f:
    chunks = pickle.load(f)

### Euler Simulation with Perturbed Parameters and Common Random Numbers

In [23]:
def EulerDiscretization_hedging(
    no_of_sim, no_of_steps, expiry,
    F_0, alpha_0, beta, rho, nu,
    h_F, h_alpha, h_nu, h_rho,
    seed=None
):
    # Time increments
    dt = expiry / no_of_steps
    dt_sqrt = cp.sqrt(dt)

    # Seed
    rng = cp.random.RandomState(seed) if seed is not None else cp.random

    # Batching to prevent GPU memory overflow
    chunk_size = 500000

    out_base  = []
    out_Fp    = []
    out_Fm    = []
    out_ap    = []
    out_am    = []
    out_nup   = []
    out_num   = []
    out_rhop  = []
    out_rhom  = []

    # Loop over the total number of simulations in batches
    for chunk_start in range(0, no_of_sim, chunk_size):

        chunk_end = min(chunk_start + chunk_size, no_of_sim)
        current_chunk_size = chunk_end - chunk_start

        # Initialize base and perturabtions
        F      = cp.full(current_chunk_size, F_0, dtype=cp.float32)
        alpha  = cp.full(current_chunk_size, alpha_0, dtype=cp.float32)
        alive  = cp.ones(current_chunk_size, dtype=cp.bool_)

        # f
        F_Fp     = cp.full(current_chunk_size, F_0 + h_F, dtype=cp.float32)
        F_Fm     = cp.full(current_chunk_size, F_0 - h_F, dtype=cp.float32)
        alpha_Fp = cp.full(current_chunk_size, alpha_0, dtype=cp.float32)
        alpha_Fm = cp.full(current_chunk_size, alpha_0, dtype=cp.float32)
        alive_Fp = cp.ones_like(alive)
        alive_Fm = cp.ones_like(alive)

        # alpha
        F_ap     = cp.full(current_chunk_size, F_0, dtype=cp.float32)
        F_am     = cp.full(current_chunk_size, F_0, dtype=cp.float32)
        alpha_ap = cp.full(current_chunk_size, alpha_0 + h_alpha, dtype=cp.float32)
        alpha_am = cp.full(current_chunk_size, alpha_0 - h_alpha, dtype=cp.float32)
        alive_ap = cp.ones_like(alive)
        alive_am = cp.ones_like(alive)

        # nu
        F_nup     = cp.full(current_chunk_size, F_0, dtype=cp.float32)
        F_num     = cp.full(current_chunk_size, F_0, dtype=cp.float32)
        alpha_nup = cp.full(current_chunk_size, alpha_0, dtype=cp.float32)
        alpha_num = cp.full(current_chunk_size, alpha_0, dtype=cp.float32)
        alive_nup = cp.ones_like(alive)
        alive_num = cp.ones_like(alive)

        # rho 
        F_rhop     = cp.full(current_chunk_size, F_0, dtype=cp.float32)
        F_rhom     = cp.full(current_chunk_size, F_0, dtype=cp.float32)
        alpha_rhop = cp.full(current_chunk_size, alpha_0, dtype=cp.float32)
        alpha_rhom = cp.full(current_chunk_size, alpha_0, dtype=cp.float32)
        alive_rhop = cp.ones_like(alive)
        alive_rhom = cp.ones_like(alive)

        def step(F, alpha, alive, rho_val, nu_val, Z1, Y1):
            dW_F = Z1
            dW_alpha = rho_val * Z1 + cp.sqrt(1 - rho_val**2) * Y1

            F_b = cp.where(alive, F ** beta, 0.0)
            F_new = F + alpha * F_b * dW_F * dt_sqrt
            alpha_new = alpha + nu_val * alpha * dW_alpha * dt_sqrt

            F = cp.where(alive, F_new, F)
            alpha = cp.where(alive, alpha_new, alpha)

            dead = (F <= 0)
            F = cp.where(dead, 0.0, F)
            alpha = cp.where(dead, 0.0, alpha)
            alive = alive & ~dead

            return F, alpha, alive

        for _ in range(no_of_steps):

            Z1 = rng.normal(size=current_chunk_size, dtype=cp.float32)
            Y1 = rng.normal(size=current_chunk_size, dtype=cp.float32)

            F, alpha, alive = step(F, alpha, alive, rho, nu, Z1, Y1)

            F_Fp, alpha_Fp, alive_Fp = step(F_Fp, alpha_Fp, alive_Fp, rho, nu, Z1, Y1)
            F_Fm, alpha_Fm, alive_Fm = step(F_Fm, alpha_Fm, alive_Fm, rho, nu, Z1, Y1)

            F_ap, alpha_ap, alive_ap = step(F_ap, alpha_ap, alive_ap, rho, nu, Z1, Y1)
            F_am, alpha_am, alive_am = step(F_am, alpha_am, alive_am, rho, nu, Z1, Y1)

            F_nup, alpha_nup, alive_nup = step(F_nup, alpha_nup, alive_nup, rho, nu + h_nu, Z1, Y1)
            F_num, alpha_num, alive_num = step(F_num, alpha_num, alive_num, rho, nu - h_nu, Z1, Y1)

            F_rhop, alpha_rhop, alive_rhop = step(F_rhop, alpha_rhop, alive_rhop, rho + h_rho, nu, Z1, Y1)
            F_rhom, alpha_rhom, alive_rhom = step(F_rhom, alpha_rhom, alive_rhom, rho - h_rho, nu, Z1, Y1)

        out_base.append(F.get())
        out_Fp.append(F_Fp.get())
        out_Fm.append(F_Fm.get())
        out_ap.append(F_ap.get())
        out_am.append(F_am.get())
        out_nup.append(F_nup.get())
        out_num.append(F_num.get())
        out_rhop.append(F_rhop.get())
        out_rhom.append(F_rhom.get())

    return (
        np.concatenate(out_base),
        np.concatenate(out_Fp),
        np.concatenate(out_Fm),
        np.concatenate(out_ap),
        np.concatenate(out_am),
        np.concatenate(out_nup),
        np.concatenate(out_num),
        np.concatenate(out_rhop),
        np.concatenate(out_rhom),
    )

### Resimulation over Training Dataset Parameters

In [24]:
def resimulate_chunks_with_greeks(chunks):
    updated_chunks = []

    for chunk in tqdm(chunks):
        alpha = chunk['alpha'].iloc[0]
        beta  = chunk['beta'].iloc[0]
        rho   = chunk['rho'].iloc[0]
        nu    = chunk['nu'].iloc[0]
        T     = chunk['T'].iloc[0]
        F0    = chunk['forward_price'].iloc[0]
        strikes = chunk['strike_price'].values

        # 0.25% bump
        h_F     = 0.0025 * F0
        h_alpha = 0.0025 * alpha
        h_nu    = 0.0025 * nu
        h_rho   = 0.0025 * rho

        # ---- Run full perturbation simulator ----
        (F_base, F_Fp, F_Fm, F_ap, F_am, F_nup, F_num, F_rhop, F_rhom) = EulerDiscretization_hedging(
            no_of_sim=2_000_000,
            no_of_steps=1400,
            expiry=T,
            F_0=F0,
            alpha_0=alpha,
            beta=beta,
            rho=rho,
            nu=nu,
            h_F=h_F,
            h_alpha=h_alpha,
            h_nu=h_nu,
            h_rho=h_rho,
            seed=42
        )

        # Prices
        def compute_call_prices(paths):
            return np.array([np.mean(np.maximum(paths - k, 0.0)) for k in strikes])
        calls_base = compute_call_prices(F_base)
        calls_Fp = compute_call_prices(F_Fp)
        calls_Fm = compute_call_prices(F_Fm)
        calls_ap = compute_call_prices(F_ap)
        calls_am = compute_call_prices(F_am)
        calls_nup = compute_call_prices(F_nup)
        calls_num = compute_call_prices(F_num)
        calls_rhop = compute_call_prices(F_rhop)
        calls_rhom = compute_call_prices(F_rhom)

        # IV
        ivs_base = vectorized_implied_volatility(calls_base, F0, strikes, T, 0, "c")
        ivs_Fp = vectorized_implied_volatility(calls_Fp,   F0, strikes, T, 0, "c")
        ivs_Fm = vectorized_implied_volatility(calls_Fm,   F0, strikes, T, 0, "c")
        ivs_ap = vectorized_implied_volatility(calls_ap,   F0, strikes, T, 0, "c")
        ivs_am = vectorized_implied_volatility(calls_am,   F0, strikes, T, 0, "c")
        ivs_nup = vectorized_implied_volatility(calls_nup,  F0, strikes, T, 0, "c")
        ivs_num = vectorized_implied_volatility(calls_num,  F0, strikes, T, 0, "c")
        ivs_rhop = vectorized_implied_volatility(calls_rhop, F0, strikes, T, 0, "c")
        ivs_rhom = vectorized_implied_volatility(calls_rhom, F0, strikes, T, 0, "c")

        # Price derivatives
        dV_dF0 = (calls_Fp - calls_Fm) / (2 * h_F)
        dV_dalpha = (calls_ap - calls_am) / (2 * h_alpha)
        dV_dnu = (calls_nup - calls_num) / (2 * h_nu)
        dV_drho = (calls_rhop - calls_rhom) / (2 * h_rho)

        # IV derivatives
        dIV_dF0 = (ivs_Fp - ivs_Fm) / (2 * h_F)
        dIV_dalpha  = (ivs_ap - ivs_am) / (2 * h_alpha)
        dIV_dnu = (ivs_nup - ivs_num) / (2 * h_nu)
        dIV_drho = (ivs_rhop - ivs_rhom) / (2 * h_rho)

        # Store
        chunk['iv_base'] = ivs_base

        chunk['calls_Fp'] = calls_Fp
        chunk['calls_Fm'] = calls_Fm
        chunk['calls_ap'] = calls_ap
        chunk['calls_am'] = calls_am
        chunk['calls_nup'] = calls_nup
        chunk['calls_num'] = calls_num
        chunk['calls_rhop'] = calls_rhop
        chunk['calls_rhom'] = calls_rhom

        chunk['ivs_Fp'] = ivs_Fp
        chunk['ivs_Fm'] = ivs_Fm
        chunk['ivs_ap'] = ivs_ap
        chunk['ivs_am'] = ivs_am
        chunk['ivs_nup'] = ivs_nup
        chunk['ivs_num'] = ivs_num
        chunk['ivs_rhop'] = ivs_rhop
        chunk['ivs_rhom'] = ivs_rhom

        chunk['dV_dF0'] = dV_dF0
        chunk['dV_dalpha'] = dV_dalpha
        chunk['dV_dnu'] = dV_dnu
        chunk['dV_drho'] = dV_drho

        chunk['dIV_dF0'] = dIV_dF0
        chunk['dIV_dalpha'] = dIV_dalpha
        chunk['dIV_dnu'] = dIV_dnu
        chunk['dIV_drho'] = dIV_drho

        chunk['h_F'] = h_F
        chunk['h_alpha'] = h_alpha
        chunk['h_nu'] = h_nu
        chunk['h_rho'] = h_rho

        updated_chunks.append(chunk)

    return updated_chunks

In [ ]:
updated_df = resimulate_chunks_with_greeks(chunks[0:1000])

# Save
output_path = "recomputed_dataset_hedging.pkl"
with open(output_path, "wb") as f:
    pickle.dump(updated_df, f)